## ***Adaptations on the Evaluation of Global Methods by Instance-based Metrics***


> Moduled: Accpeting the four GNNs (GCN+GAP, DGCNN, DIFFPOOL, and GIN)


---

In [ ]:
"""
The first algorithm:
1. find average size of explanation in each class by GraphMask in terms of number of edges.
2. generate explanations for that count of edges in XGNN and GNNInterpreter.
3. check existance of important edges from each class in each sample.

"""


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install git+https://github.com/rusty1s/pytorch_geometric.git

2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 59.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/rusty1s/pytorch_geometric.git to /tmp/pip-req-build-18wyjdv0
  Running command git clone --filter=blob:none --quiet https://github.com/rusty1s/pytorch_geometric.git /tmp/pip-req-build-18wyjdv0
  Resolved https://github.com/rusty1s/pytorch_geometric.git to commit 36bc9255901fa4afaf03d55fec913ec56888ed53
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from math import sqrt
import math
from torch_geometric.datasets import TUDataset
import torch as th
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn.parameter import Parameter
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear, LayerNorm
from sklearn import metrics
from scipy.spatial.distance import hamming
import statistics
import pandas
from time import perf_counter
from IPython.core.display import deepcopy
from torch_geometric.nn import MessagePassing
import copy
from torch.nn import ReLU, Sequential
from torch import sigmoid
from itertools import chain
from time import perf_counter
from torch_geometric.data import Data, Batch, Dataset
from functools import partial
from torch_geometric.utils import to_networkx
from torch_geometric.utils import remove_self_loops
from typing import Callable, Union, Optional
#from torch_geometric.utils.num_nodes import maybe_num_nodes
import networkx as nx
from typing import List, Tuple, Dict
from collections import Counter
import statistics
import tqdm
import csv
from statistics import mean
from torch_geometric.utils import from_scipy_sparse_matrix
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
import torch_geometric.nn as gnn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mutag_dataset = TUDataset(root='data/TUDataset', name='MUTAG')

In [ ]:
Explainability_name = 'TopDown Approach'
Task_name = 'Graph Classification'
checkpoint_directory_Classifier = "/content/drive/My Drive/Explainability Methods/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/model_classifier.pt"
classifier_lr = 0.001
classifier_dropout = 0.1
classifier_weight_decay = 1e-6
classifier_bias = True
DataSet_name = "MUTAG"

#File_Name = Model_name + " " + Explainability_name + " " + Task_name + " " + DataSet_name + " "

In [ ]:
#train_dataset, test_dataset = train_test_split(dataset, test_size=0.3, random_state=0, shuffle=True)
#print("Number of Training Graphs: ", len(train_dataset))
#print("Number of Test Graphs: ", len(test_dataset))

df = pandas.read_csv("/content/drive/My Drive/Explainability Methods/Train and Test Indexes on Graph Classification/Experimental Results/train_test_indexes.csv")

read_training_list_indexes__ = df['Train Indexes']
read_test_list_indexes__ = df['Test Indexes']
read_test_list_indexes__ = read_test_list_indexes__.dropna()
read_test_list_indexes = []
read_training_list_indexes = []
for element in read_test_list_indexes__:
    read_test_list_indexes.append(int(element))
for element in read_training_list_indexes__:
    read_training_list_indexes.append(int(element))


print(read_training_list_indexes)
print(read_test_list_indexes)

mutag_train_dataset = []
mutag_test_dataset = []
for index in read_training_list_indexes:
    mutag_train_dataset.append(mutag_dataset[index])
for index in read_test_list_indexes:
    mutag_test_dataset.append(mutag_dataset[index])


print(f'Number of training graphs: {len(mutag_train_dataset)}')
print(f'Number of test graphs: {len(mutag_test_dataset)}')

[51, 142, 143, 10, 4, 141, 125, 23, 3, 79, 28, 117, 136, 156, 83, 128, 109, 70, 31, 58, 7, 148, 14, 187, 20, 162, 123, 13, 91, 185, 169, 102, 32, 55, 119, 25, 161, 175, 52, 121, 152, 108, 113, 65, 39, 103, 131, 42, 166, 110, 160, 68, 90, 89, 64, 172, 159, 72, 170, 18, 122, 29, 179, 49, 171, 178, 9, 74, 96, 48, 181, 127, 126, 87, 12, 163, 88, 53, 94, 146, 5, 158, 16, 67, 6, 59, 164, 151, 34, 47, 54, 46, 100, 112, 93, 182, 66, 106, 124, 19, 186, 133, 45, 15, 40, 167, 174, 98, 105, 153, 61, 63, 132, 116, 43, 80, 33, 147, 165, 69, 135, 86, 76, 57, 173, 115, 138, 140, 134, 180, 95, 22, 38, 41, 24, 120, 145, 26, 21, 50, 176, 107, 78, 17, 85, 154, 60, 92, 184, 129]
[0, 1, 2, 8, 11, 27, 30, 35, 36, 37, 44, 56, 62, 71, 73, 75, 77, 81, 82, 84, 97, 99, 101, 104, 111, 114, 118, 130, 137, 139, 144, 149, 150, 155, 157, 168, 177, 183]
Number of training graphs: 150
Number of test graphs: 38


In [ ]:
class global_explanation_and_samples_intersection:
    def __init__(self, explanation, input_graph):
        self.explanation = explanation
        self.input_graph = input_graph

    def pair_source_and_target_node_features(self, edges, node_feats):
        paired_feat = {}
        for (source_index, target_index) in edges:
            paired_feat[(source_index, target_index)] = node_feats[source_index] + node_feats[target_index]
        return paired_feat

    def find_common_edges(self, ex_graph_node_features, ex_graph_edge_index, input_graph_node_features, input_graph_edge_index):

        ex_graph_edges = [(source1_index, target1_index) for source1_index, target1_index in ex_graph_edge_index.T.tolist()]
        input_graph_edges = [(source2_index, target2_index) for source2_index, target2_index in input_graph_edge_index.T.tolist()]

        ex_graph_paired_node_features = self.pair_source_and_target_node_features(ex_graph_edges, ex_graph_node_features)
        input_graph_paired_node_features = self.pair_source_and_target_node_features(input_graph_edges, input_graph_node_features)

        common_edges = {}
        for (inp_graph_source_index, inp_graph_target_index), inp_graph_paired_feat in input_graph_paired_node_features.items():
            presence = 0
            for (ex_graph_source_index, ex_graph_target_index), ex_graph_paired_feat in ex_graph_paired_node_features.items():
                if inp_graph_paired_feat == ex_graph_paired_feat:
                    presence = 1
            if presence == 1:
                common_edges[(inp_graph_source_index, inp_graph_target_index)] = 1
            else:
                common_edges[(inp_graph_source_index, inp_graph_target_index)] = 0

        return common_edges
    def __call__(self):
        return self.find_common_edges(self.explanation.x.tolist(), self.explanation.edge_index, self.input_graph.x.tolist(), self.input_graph.edge_index)




common_edges_finder = global_explanation_and_samples_intersection(explanation=mutag_dataset[0], input_graph=mutag_dataset[7])
intersection_edges = common_edges_finder()

for key, value in intersection_edges.items():
    print("key: ", key, "   value: ", value)

key:  (0, 1)    value:  1
key:  (1, 0)    value:  1
key:  (1, 2)    value:  0
key:  (1, 3)    value:  0
key:  (2, 1)    value:  0
key:  (3, 1)    value:  0
key:  (3, 4)    value:  0
key:  (4, 3)    value:  0
key:  (4, 5)    value:  1
key:  (4, 9)    value:  1
key:  (5, 4)    value:  1
key:  (5, 6)    value:  1
key:  (6, 5)    value:  1
key:  (6, 7)    value:  1
key:  (7, 6)    value:  1
key:  (7, 8)    value:  1
key:  (7, 12)    value:  1
key:  (8, 7)    value:  1
key:  (8, 9)    value:  1
key:  (8, 10)    value:  1
key:  (9, 4)    value:  1
key:  (9, 8)    value:  1
key:  (10, 8)    value:  1
key:  (10, 11)    value:  1
key:  (11, 10)    value:  1
key:  (11, 12)    value:  1
key:  (11, 16)    value:  1
key:  (12, 7)    value:  1
key:  (12, 11)    value:  1
key:  (12, 13)    value:  1
key:  (13, 12)    value:  1
key:  (13, 14)    value:  1
key:  (14, 13)    value:  1
key:  (14, 15)    value:  1
key:  (15, 14)    value:  1
key:  (15, 16)    value:  1
key:  (15, 17)    value:  1
key:  (1

In [ ]:
sizes = []
for graph in mutag_dataset:
    sizes.append(len(graph.x))

In [ ]:
print(sizes.index(max(sizes)))
print(sizes.index(min(sizes)))

5
75
